In [1]:
import pandas as pd
import numpy as np
from cassandra.cluster import Cluster
from geopy.distance import distance
import math

# Falta saber si son poligonos en un plano o puntos en un globo 

### Prueba consulta de restaurantes 

In [2]:
with Cluster() as cluster:
    with cluster.connect('challenge') as session:
        result = session.execute(
            'select * from restaurants limit 10')

In [3]:
for item in result:
    print(item)

Row(id='55cba2476c522cafdb058fdf', location=[40.7092884, -73.8048161], name='The Smile Of The Beyond', type='Point')
Row(id='55cba2476c522cafdb05821e', location=[40.63628, -74.009073], name="Mr.Q'S Gourmet", type='Point')
Row(id='55cba2476c522cafdb054d62', location=[40.7076928, -74.00208219999999], name="Jeremy'S Ale House", type='Point')
Row(id='55cba2476c522cafdb05470a', location=[40.7430799, -73.91466319999999], name='Boston Market', type='Point')
Row(id='55cba2476c522cafdb056cd4', location=[40.8774338, -73.90678129999999], name='Pick Up Six Fresh Fast Asian Kitchen', type='Point')
Row(id='55cba2476c522cafdb055856', location=[40.7557186, -73.98073339999999], name='Freefoods', type='Point')
Row(id='55cba2476c522cafdb0574e5', location=[40.7791655, -73.9629278], name='Metropolitan Museum Roof Top Garden Cafe', type='Point')
Row(id='55cba2476c522cafdb056814', location=[40.7140481, -73.9989528], name='Lobster Boat Restaurant', type='Point')
Row(id='55cba2486c522cafdb0598a4', location=[40

### Extraer un vecindario tipo MultiPolygono
* 55cb9c666c522cafdb053a97

In [4]:
with Cluster() as cluster:
    with cluster.connect('challenge') as session:
        result = session.execute(
            'select * from neighborhoods where id = \'55cb9c666c522cafdb053a97\'')
        length = session.execute(
            'select count(*) from neighborhoods where id = \'55cb9c666c522cafdb053a97\'')

In [5]:
print('Puntos extraidos: '+str(length[0][0]) + '\n\nImprime puntos')
for row in result:
    print(row)

Puntos extraidos: 271

Imprime puntos
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=0, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.73311366098218, -73.96230391870556])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=1, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.7329155516237, -73.96226340489365])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=2, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.732913300829765, -73.96236043096485])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=3, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.73287557416924, -73.96285045581647])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=4, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.73283805163722, -73.96332992178779])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=5, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.73284587606134,

### Encontrar restaurantes en una distancia específica respecto a un punto concreto
* Distancia = 1km
* Punto = [-73.94378455587042, 40.6973697290538]

In [9]:
distancia = 1
puntoInicial = [40.6973697290538, -73.94378455587042]

In [22]:
with Cluster() as cluster:
    with cluster.connect('challenge') as session:
        result = session.execute('select * from restaurants')
        filtrado = []
        for row in result:
            if distance(puntoInicial, row[1]).km <= distancia:
                # if row[1][0] > 90 or row[1][0] < -90 or row[1][1] > 180 or row[1][1] < -180:
                filtrado.append([row[0], row[1], row[2], round(
                    distance(puntoInicial, row[1]).km, 3)])

In [25]:
def sortFunc(item): return item[3]

In [26]:
print('Numero de restaurantes dentro del límite',
      len(filtrado), f'\n\nImprimir restaurantes a distancia {distancia} desde {puntoInicial}:')
filtrado.sort(key=sortFunc)
for item in filtrado:
    print(item, '<- Distancia km')

Numero de restaurantes dentro del límite 124 

Imprimir restaurantes a distancia 1 desde [40.6973697290538, -73.94378455587042]:
['55cba2476c522cafdb055b91', [40.698466, -73.944343], 'Migdalia Place', 0.131] <- Distancia km
['55cba2476c522cafdb057349', [40.695837, -73.943679], 'Elisa Cafe', 0.17] <- Distancia km
['55cba2476c522cafdb057cf8', [40.695896, -73.942544], 'Crown Fried Chicken', 0.194] <- Distancia km
['55cba2476c522cafdb057b12', [40.699517, -73.944503], 'Genao', 0.246] <- Distancia km
['55cba2476c522cafdb05613a', [40.699474, -73.94279399999999], 'Towers Cafe', 0.248] <- Distancia km
['55cba2486c522cafdb059523', [40.6961798, -73.9467093], 'Crown Fried Chicken', 0.28] <- Distancia km
['55cba2476c522cafdb05465a', [40.695877, -73.946589], "Mike'S Pizza", 0.289] <- Distancia km
['55cba2476c522cafdb0569af', [40.695867, -73.94667299999999], 'China City Restaurant', 0.296] <- Distancia km
['55cba2476c522cafdb054cb8', [40.698919, -73.9409297], 'Peking Kitchen', 0.296] <- Distancia km
